### TP Example
- Buy 1 BTC @ 10,000 (\$)
- SL @ 9,800

### TP Levels:
  - TP0 (SL): 9,800
  - TP1: 10,100
  - TP2: 10,200
  - TP3: 10,400
  - TP4: 10,600

### Profit Levels:
  - TP0: (9,800 - 10,000) * 100% = __- \$200__
  - TP1: (10,000 - 10,000) * 25% + (9,800 - 10,000) * 75% = __- \$125__
  - TP2: (10,100 - 10,000) * 25% + (10,200 - 10,000) * 25% = __+ \$75__
  - TP3: (10,100 - 10,000) * 25% + (10,200 - 10,000) * 25% + (10,400 - 10,000) * 25% = __+ \$175__
  - TP4: (10,100 - 10,000) * 25% + (10,200 - 10,000) * 25% + (10,400 - 10,000) * 25%  + (10,600s - 10,000) * 25% = __+ \$275__

### Risk vs. Reward
- risk = 200 + 125 = __\$325__
- reward = 75 + 175 + 275 = __\$525__
- reward/risk = 525 / 326 = __1.61538__

### Profit compared to starting SL (\$200)
  - TP0: 200 / -200 = __- 1__
  - TP1: 200 / 125 =  __- 5/8__
  - TP2: 200 / 75 = __+ 3/8__
  - TP3: 200 / 175 = __+ 7/8__
  - TP4: 200 / 275 = __+ 11/8__


In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import os

In [3]:
df = pd.read_csv(os.getcwd() + '/BTC.csv').drop('volume', axis=1)
_close = np.array(df['close'])
_open = np.array(df['open'])
low = np.array(df['low'])
high = np.array(df['high'])

print('Start Date: ', df['date'].iloc[0])
print('Start Price: ', df['open'].iloc[0])
print('End Date: ', df['date'].iloc[-1])
print('End Price: ', df['open'].iloc[-1])

Start Date:  2018-01-01 00:00:00.000
Start Price:  13539.0
End Date:  2019-07-08 15:00:00.000
End Price:  12185.57


In [4]:
# This section of code adds a 'Long' or 'Short' signal on the date it's triggered
ema3 = df['close'].ewm(span=3, adjust=False).mean()
ma20 = df['close'].rolling(window=20).mean().fillna(0)
ema40 = df['close'].ewm(span=40, adjust=False).mean()
ema3_gt_ma20 = ema3 > ma20

cross_indices = []
current_val = ema3_gt_ma20[0]

signals = [None for i in range(len(df))]

# Find intersections between ema3 and ma20
for i, val in ema3_gt_ma20[1:].items():
    if val != current_val:
        cross_indices.append(i)
    current_val = val

# Loop through intersection points
for cross_index in cross_indices:
    # Look at all points after intersection
    for index in range(cross_index, len(df)):
        rng = set(ema3_gt_ma20[cross_index:index+1])
        # End loop if ema3 has another intersection with ma20
        if len(rng) == 2:
            break
        # End loop if the size of the body of the candle is more than 2%
        elif abs(_close[index] - _open[index])/_close[index] > .02:
            break
        elif True in rng:
            if _close[index] > ema3[index]:
                if ma20[index] > ema40[index]:
                    signals[index] = 'Long'
                break
        else:   # False in rng
            if _close[index] < ema3[index]:
                if ma20[index] < ema40[index]:
                    signals[index] = 'Short'
                break

df['signal'] = signals
print('\nFirst 5 signals')
df.dropna().head()


First 5 signals


,date,open,high,low,close,signal
22,2018-01-01 22:00:00.000,13353.78,13480.84,12890.02,13127.31,Short
57,2018-01-03 09:00:00.000,14812.97,15033.77,14790.13,15033.77,Long
63,2018-01-03 15:00:00.000,14939.11,14990.00,14830.00,14926.96,Long
65,2018-01-03 17:00:00.000,14792.32,14940.82,14710.00,14919.51,Long
84,2018-01-04 13:00:00.000,14805.01,14988.00,14720.05,14895.19,Long


In [47]:
# These are from above
tp_pcts = {
    0: -1,
    1: -5./8.,
    2: 3./8.,
    3: 7./8.,
    4: 11./8.
}

results = [None for i in range(len(df))]
purchase_prices = [None for i in range(len(df))]
stop_losses = [None for i in range(len(df))]

## Scenario 1: Set SL .25% below local low

In [53]:
# Loop through signals
for signal_index, signal in enumerate(signals):
    if signal is None:
        continue
    
    if signal == 'Long':
        l_bounds = low
        midrange = _open
        u_bounds = high
        SL_cushion = .0025
    else:  # signal == 'Short'
        l_bounds = -high
        midrange = -_open
        u_bounds = -low
        SL_cushion = -.0025

    purchase_price = midrange[signal_index+1]
    purchase_prices[signal_index] = purchase_price
    
# --------------------------------------------------------------------------
# This is the only line that changes
    SL = min(l_bounds[signal_index-10:signal_index]) * (1. - SL_cushion)    
# --------------------------------------------------------------------------    
    
    stop_losses[signal_index] = SL

    diff = purchase_price - SL

    tp1 = purchase_price + diff/2.
    tp2 = purchase_price + diff
    tp3 = purchase_price + diff*2.
    tp4 = purchase_price + diff*3.

    tp_targets = iter([tp1, tp2, tp3, tp4])
    tp_target = next(tp_targets, None)
    result = 0

    for i in range(signal_index, len(df)):
        if result == 4 or midrange[i] < SL or l_bounds[i] < SL:
            break

        while result != 4 and u_bounds[i] > tp_target:
            result += 1
            tp_target = next(tp_targets, None)

        if result == 2:
            SL = purchase_price

    results[signal_index] = result


df['result'] = results
df['purchase-price'] = purchase_prices
df['stop-loss'] = stop_losses
df['stop-loss %'] = abs(df['purchase-price'] - df['stop-loss']) / df['purchase-price']

end_pct =  list(map(lambda x: tp_pcts[x], df.dropna()['result']))
print('Percent increase from starting amount: {0:.2f}'.format(sum(end_pct * df.dropna()['stop-loss %'])))

Percent increase from starting amount: 0.13


## Scenario 2: Set SL the same as local low

In [54]:
# Loop through signals
for signal_index, signal in enumerate(signals):
    if signal is None:
        continue
    
    if signal == 'Long':
        l_bounds = low
        midrange = _open
        u_bounds = high
        SL_cushion = .0025
    else:  # signal == 'Short'
        l_bounds = -high
        midrange = -_open
        u_bounds = -low
        SL_cushion = -.0025

    purchase_price = midrange[signal_index+1]
    purchase_prices[signal_index] = purchase_price
    
# --------------------------------------------------------------------------
# This is the only line that changes
    SL = min(l_bounds[signal_index-10:signal_index])
# --------------------------------------------------------------------------

    stop_losses[signal_index] = SL

    diff = purchase_price - SL

    tp1 = purchase_price + diff/2.
    tp2 = purchase_price + diff
    tp3 = purchase_price + diff*2.
    tp4 = purchase_price + diff*3.

    tp_targets = iter([tp1, tp2, tp3, tp4])
    tp_target = next(tp_targets, None)
    result = 0

    for i in range(signal_index, len(df)):
        if result == 4 or midrange[i] < SL or l_bounds[i] < SL:
            break

        while result != 4 and u_bounds[i] > tp_target:
            result += 1
            tp_target = next(tp_targets, None)

        if result == 2:
            SL = purchase_price

    results[signal_index] = result


df['result'] = results
df['purchase-price'] = purchase_prices
df['stop-loss'] = stop_losses
df['stop-loss %'] = abs(df['purchase-price'] - df['stop-loss']) / df['purchase-price']

end_pct =  list(map(lambda x: tp_pcts[x], df.dropna()['result']))
print('Percent increase from starting amount: {0:.2f}'.format(sum(end_pct * df.dropna()['stop-loss %'])))

Percent increase from starting amount: 0.18


## Scenario 3: Set SL .25% above local low

In [55]:
# Loop through signals
for signal_index, signal in enumerate(signals):
    if signal is None:
        continue
    
    if signal == 'Long':
        l_bounds = low
        midrange = _open
        u_bounds = high
        SL_cushion = .0025
    else:  # signal == 'Short'
        l_bounds = -high
        midrange = -_open
        u_bounds = -low
        SL_cushion = -.0025

    purchase_price = midrange[signal_index+1]
    purchase_prices[signal_index] = purchase_price

# --------------------------------------------------------------------------
# This is the only line that changes
    SL = min(l_bounds[signal_index-10:signal_index]) * (1. + SL_cushion)
# --------------------------------------------------------------------------
    
    stop_losses[signal_index] = SL

    diff = purchase_price - SL

    tp1 = purchase_price + diff/2.
    tp2 = purchase_price + diff
    tp3 = purchase_price + diff*2.
    tp4 = purchase_price + diff*3.

    tp_targets = iter([tp1, tp2, tp3, tp4])
    tp_target = next(tp_targets, None)
    result = 0

    for i in range(signal_index, len(df)):
        if result == 4 or midrange[i] < SL or l_bounds[i] < SL:
            break

        while result != 4 and u_bounds[i] > tp_target:
            result += 1
            tp_target = next(tp_targets, None)

        if result == 2:
            SL = purchase_price

    results[signal_index] = result


df['result'] = results
df['purchase-price'] = purchase_prices
df['stop-loss'] = stop_losses
df['stop-loss %'] = abs(df['purchase-price'] - df['stop-loss']) / df['purchase-price']

end_pct =  list(map(lambda x: tp_pcts[x], df.dropna()['result']))
print('Percent increase from starting amount: {0:.2f}'.format(sum(end_pct * df.dropna()['stop-loss %'])))

Percent increase from starting amount: 0.34


# Conclusion: Set SL above local low for higher returns